# Mount Drive to Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/Shareddrives/663-SoftwareAnalytics/Codebase/Brij_word2vec

/content/drive/Shareddrives/663-SoftwareAnalytics/Codebase/Brij_word2vec


# Importing Python Library

In [ ]:
%load_ext autoreload
%autoreload 2

import ast
import glob
import re
from pathlib import Path

import astor
import pandas as pd
import spacy
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split


EN = spacy.load('en')

# Data Parsing from database

In [ ]:

#hide

path = Path('.')
#collapse_show
def jsonl_list_to_dataframe(file_list, columns=None):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f,
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

def get_dfs(path):
    """Grabs the different data splits and converts them into dataframes"""
    dfs = []
    for split in ["train", "valid", "test"]:
        files = sorted((path/split).glob("**/*.gz"))
        df = jsonl_list_to_dataframe(files, ["code", "docstring","code_tokens","docstring_tokens","url"])
        dfs.append(df)
        
    return dfs


def tokenize_docstring(text):
    "Apply tokenization using spacy to docstrings."
    tokens = EN.tokenizer(text)
    return [token.text.lower() for token in tokens if not token.is_space]


def tokenize_code(text):
    "A very basic procedure for tokenizing code strings."
    return RegexpTokenizer(r'\w+').tokenize(text)

df_trn, df_val, df_tst = get_dfs(path/"python/final/jsonl")
df_trn.head()


,code,docstring,code_tokens,docstring_tokens,url
0,"def train(train_dir, model_save_path=None, n_n...",Trains a k-nearest neighbors classifier for fa...,"[def, train, (, train_dir, ,, model_save_path,...","[Trains, a, k, -, nearest, neighbors, classifi...",https://github.com/ageitgey/face_recognition/b...
1,"def predict(X_img_path, knn_clf=None, model_pa...",Recognizes faces in given image using a traine...,"[def, predict, (, X_img_path, ,, knn_clf, =, N...","[Recognizes, faces, in, given, image, using, a...",https://github.com/ageitgey/face_recognition/b...
2,"def show_prediction_labels_on_image(img_path, ...",Shows the face recognition results visually.\n...,"[def, show_prediction_labels_on_image, (, img_...","[Shows, the, face, recognition, results, visua...",https://github.com/ageitgey/face_recognition/b...
3,"def _rect_to_css(rect):\n """"""\n Convert ...",Convert a dlib 'rect' object to a plain tuple ...,"[def, _rect_to_css, (, rect, ), :, return, rec...","[Convert, a, dlib, rect, object, to, a, plain,...",https://github.com/ageitgey/face_recognition/b...
4,"def _trim_css_to_bounds(css, image_shape):\n ...","Make sure a tuple in (top, right, bottom, left...","[def, _trim_css_to_bounds, (, css, ,, image_sh...","[Make, sure, a, tuple, in, (, top, right, bott...",https://github.com/ageitgey/face_recognition/b...


# Sumsampling the database and generate word2vec Embedding

In [ ]:
#Subsample
df_trn_sub = df_trn[:5000]
docstring = list(df_trn_sub['docstring'].values)
code = list(df_trn_sub['code'].values)
codeToken = []
docToken = []

for singleCode,singleDoc in tqdm(zip(code,docstring)):
  codeToken.append(' '.join(tokenize_code(singleCode)))
  docToken.append(' '.join(tokenize_docstring(singleDoc)))
df_trn_sub['code token'] = codeToken
df_trn_sub['doc token'] = docToken
code_token = list(df_trn_sub['code_tokens'].values)

code_token_list = []
for code in code_token:
  code_token_list.append(' '.join(code))

import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("","",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

#Cleaning the code tokens
df_idf = pd.DataFrame()
df_idf['text'] = code_token_list
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))
clean_code = list(df_idf['text'].values)
clean_code_token = []
for code in clean_code:
  clean_code_token.append(code.split(' '))
df_trn_sub['clean_code'] = clean_code_token

#Generate the code and doc embeddings
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
data_corpus = list(df_trn_sub.clean_code.values)
document_corpus = list(df_trn_sub.docstring.values)
buffer = []
for data in data_corpus:buffer.append(' '.join(data[:200]))
vocabulary=vectorizer.fit(buffer)
X= vectorizer.transform(buffer)
baseData = X.toarray()
all_embeddings = np.array(baseData)


5000it [00:03, 1534.47it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [ ]:

import numpy as np
from scipy.spatial import distance
from sklearn.metrics import ndcg_score
import numpy as np
from scipy.spatial import distance

testDocstring = list(df_tst.docstring.values)
filteredDocstring = []
for doc in testDocstring:
  buffer = doc.split(' ')
  #if len(buffer) < 10:filteredDocstring.append(doc)
  filteredDocstring.append(doc)
filteredDocstring = filteredDocstring[:400]

In [ ]:
print(filteredDocstring[:5],len(filteredDocstring))

['Extracts video ID from URL.', 'str->list\n    Convert XML to URL List.\n    From Biligrab.', 'From http://cdn37.atwikiimg.com/sitescript/pub/dksitescript/FC2.site.js\n    Also com.hps.util.fc2.FC2EncrptUtil.makeMimiLocal\n    L110', 'wrapper', 'Downloads Dailymotion videos by URL.'] 400


In [ ]:

from sklearn.metrics.pairwise import cosine_similarity
closest_doc_indexes = {}
testingVector = vectorizer.transform(filteredDocstring)
doctrainingVector = vectorizer.transform(document_corpus)
for index, test_doc_embd in tqdm(enumerate(testingVector)):
  test_doc_embd = np.reshape(test_doc_embd,(1,-1))
  similarities = cosine_similarity(test_doc_embd,doctrainingVector)[0]
  
  most_similar_train_doc = np.argmax(similarities)
  closest_doc_indexes[index] = most_similar_train_doc

print(closest_doc_indexes)


400it [00:01, 229.69it/s]

{0: 3683, 1: 3992, 2: 4205, 3: 4198, 4: 4445, 5: 3286, 6: 3731, 7: 3731, 8: 4398, 9: 4398, 10: 4445, 11: 4198, 12: 3979, 13: 2107, 14: 1750, 15: 3683, 16: 3971, 17: 3959, 18: 1734, 19: 3765, 20: 2372, 21: 2107, 22: 3942, 23: 3868, 24: 3400, 25: 4293, 26: 2659, 27: 3683, 28: 4293, 29: 737, 30: 440, 31: 3183, 32: 4445, 33: 710, 34: 3816, 35: 3327, 36: 2140, 37: 4741, 38: 3343, 39: 1137, 40: 4574, 41: 1999, 42: 2394, 43: 2394, 44: 1065, 45: 1131, 46: 3942, 47: 2107, 48: 3805, 49: 3099, 50: 3805, 51: 3031, 52: 2243, 53: 1953, 54: 2351, 55: 4574, 56: 3782, 57: 3592, 58: 3031, 59: 636, 60: 703, 61: 715, 62: 3282, 63: 632, 64: 366, 65: 1717, 66: 3782, 67: 703, 68: 3572, 69: 664, 70: 3543, 71: 3513, 72: 4864, 73: 3259, 74: 3259, 75: 3259, 76: 3820, 77: 3587, 78: 1750, 79: 3357, 80: 673, 81: 418, 82: 407, 83: 673, 84: 158, 85: 673, 86: 673, 87: 673, 88: 407, 89: 972, 90: 3282, 91: 3285, 92: 3441, 93: 695, 94: 1842, 95: 1842, 96: 3285, 97: 156, 98: 3466, 99: 3296, 100: 3751, 101: 3744, 102: 3282

In [ ]:
import pickle
with open('saved_dictionary.pkl', 'wb') as f:
    pickle.dump(closest_doc_indexes, f)
        
with open('saved_dictionary.pkl', 'rb') as f:
        loaded_dict = pickle.load(f)

In [ ]:

from tqdm import tqdm
result = []
ndcga = []
mrr = []
for doc in tqdm(filteredDocstring):
    testingString = [doc]
    testingVector = vectorizer.transform(testingString)
    testingVector = testingVector.toarray()
    distances = distance.cdist(testingVector, baseData, "cosine")[0]
    min_index = np.argmin(distances)
    min_distance = distances[min_index]
    max_similarity = 1 - min_distance
    result.append(max_similarity)
    ndcga.append((ndcg_score(np.asarray(testingVector), np.asarray([baseData[min_index]]))))
    index = filteredDocstring.index(doc)
    closest = closest_doc_indexes[index]
    best = np.argsort(distances)[:50]
    if closest in best:
      best = list(best)
      mrr.append(1/(best.index(closest)+1))
    else:
      mrr.append(0)

100%|██████████| 400/400 [02:44<00:00,  2.43it/s]


In [ ]:
result = [x for x in result if str(x) != 'nan']

print('NDCGA Results : ',sum(ndcga)/len(ndcga))
print('Similarity Average : ',sum(result)/len(result))
print('Mean Reciprocal Rank :  ',sum(mrr)/len(mrr))

0.5448739729695309
0.40063446923237145
0.0220853657129706
